# Tool Integration and Custom Tools - Interactive Notebook

This notebook provides hands-on examples to accompany the tools lesson:
- Design and implement custom tools
- Validate inputs with Pydantic and StructuredTool
- Integrate external APIs and local resources
- Implement async tools for concurrency
- Add retries, timeouts, and observability
- Compose tools in OpenAI tool-calling agents

Ensure your `.env` contains required API keys (e.g., OPENAI_API_KEY, NEWSAPI_KEY).

## 0) Setup and Imports

In [ ]:
import os
import time
import json
from typing import Optional, Dict, Any, List

from dotenv import load_dotenv
load_dotenv()

# LangChain core
from langchain_core.tools import tool, StructuredTool
from langchain_core.prompts import ChatPromptTemplate

# LLM and agent utilities
from langchain_openai import ChatOpenAI

# Agent constructors (version-dependent)
agents_available = True
try:
    from langchain.agents import create_openai_tools_agent, AgentExecutor
except Exception as e:
    print("Agent constructors not available from langchain.agents in this version.")
    print("Error:", e)
    agents_available = False

print("Setup complete.")

## 1) Quick Function Tools with @tool

In [ ]:
from math import isfinite

@tool
def calculator(expression: str) -> str:
    """Evaluate a basic arithmetic expression like '2 + 3 * 4'."""
    try:
        # WARNING: eval is unsafe for untrusted input. Use a proper expression parser in production.
        result = eval(expression, {"__builtins__": {}}, {})
        if isinstance(result, (int, float)) and isfinite(result):
            return str(result)
        return "calc-error: non-finite result"
    except Exception as e:
        return f"calc-error: {e}"

print("calculator tool ready.")
print("calculator 2+2 =>", calculator.func("2+2"))

## 2) Typed Tools with Pydantic Validation (StructuredTool)

In [ ]:
from pydantic import BaseModel, Field

class WeatherInput(BaseModel):
    city: str = Field(..., description="City name, e.g., 'London'")
    units: str = Field("metric", description="'metric' or 'imperial'")

# Mock implementation; replace with a real API call as needed
def fetch_weather(city: str, units: str = "metric") -> dict:
    if not city or len(city) < 2:
        raise ValueError("city too short")
    # Return small typed payload; agent composes prose
    return {"city": city, "units": units, "temp": 24.0}

weather_tool = StructuredTool.from_function(
    func=fetch_weather,
    name="weather",
    description="Get current weather for a city.",
    args_schema=WeatherInput
)

print("weather tool ready.")
print(weather_tool.invoke({"city": "London", "units": "metric"}))

## 3) External API Tool (NewsAPI.org) with Requests and Validation

In [ ]:
import requests

class NewsInput(BaseModel):
    query: str = Field(..., description="Topic to search")
    max_results: int = Field(3, ge=1, le=10)

def search_news(query: str, max_results: int = 3) -> List[dict]:
    api_key = os.getenv("NEWSAPI_KEY")
    if not api_key:
        return [{"error": "missing NEWSAPI_KEY"}]
    url = "https://newsapi.org/v2/everything"
    params = {"q": query, "pageSize": max_results, "sortBy": "relevancy", "apiKey": api_key}
    try:
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()
        articles = data.get("articles", [])[:max_results]
        return [{"title": a.get("title", ""), "url": a.get("url", "")} for a in articles]
    except Exception as e:
        return [{"error": str(e)}]

news_tool = StructuredTool.from_function(
    func=search_news,
    name="news_search",
    description="Search recent news via NewsAPI.org",
    args_schema=NewsInput
)

print("news_search tool ready.")
print(news_tool.invoke({"query": "AI", "max_results": 2}))

## 4) Local Resource Tool (Safe File Reader)

In [ ]:
from pathlib import Path

@tool
def read_local_file(path: str, max_bytes: int = 4000) -> str:
    """Read a small local text file; caps bytes to prevent large loads."""
    p = Path(path)
    if not p.exists() or not p.is_file():
        return "read-error: file not found"
    data = p.read_bytes()[:max_bytes]
    try:
        return data.decode("utf-8", errors="replace")
    except Exception:
        return "read-error: decode failure"

# Create a sample file for demo
Path("./notes").mkdir(exist_ok=True)
Path("./notes/sample.txt").write_text("Vector memory helps recall facts via embeddings.")
print(read_local_file.func("./notes/sample.txt"))

## 5) Async Tool (aiohttp) for Concurrency

In [ ]:
import asyncio
try:
    import aiohttp
    aio_available = True
except Exception as e:
    print("aiohttp not available:", e)
    aio_available = False

class HttpGetInput(BaseModel):
    url: str = Field(..., description="HTTP/HTTPS URL")

async def http_get_async(url: str) -> dict:
    timeout = aiohttp.ClientTimeout(total=10)
    async with aiohttp.ClientSession(timeout=timeout) as session:
        try:
            async with session.get(url) as resp:
                text = await resp.text()
                return {"status": resp.status, "length": len(text)}
        except Exception as e:
            return {"error": str(e)}

if aio_available:
    http_get_tool = StructuredTool.from_function(
        func=http_get_async,
        name="http_get",
        description="Fetch a URL asynchronously and return status and length",
        args_schema=HttpGetInput,
        coroutine=http_get_async
    )
    print("http_get tool ready.")
    # Run a quick async demo
    async def _demo():
        return await http_get_async("https://example.com")
    print("async demo:", asyncio.run(_demo()))
else:
    print("Skipping async tool demo; aiohttp unavailable.")

## 6) Retries, Backoff, and Timeouts (Decorator)

In [ ]:
from functools import wraps

def with_retry(retries=3, backoff=0.5):
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            last = None
            for i in range(retries):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    last = e
                    time.sleep(backoff * (2**i))
            return f"retry-error: {last}"
        return wrapper
    return deco

@tool
@with_retry(retries=2, backoff=0.2)
def fragile_operation(x: int) -> str:
    """Sometimes fails; returns x squared on success."""
    if x % 2:
        raise RuntimeError("simulated intermittent error")
    return str(x * x)

print("fragile_operation tool ready.")
print("fragile_operation(2) =>", fragile_operation.func(2))
print("fragile_operation(3) =>", fragile_operation.func(3))

## 7) Compose Tools in OpenAI Tool-Calling Agent

In [ ]:
llm = None
try:
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
except Exception as e:
    print("ChatOpenAI unavailable (check OPENAI_API_KEY):", e)

tools_for_agent = [calculator, weather_tool, news_tool, read_local_file]
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a precise assistant. Prefer tools when helpful."),
    ("human", "{input}")
])

if agents_available and llm is not None:
    try:
        agent = create_openai_tools_agent(llm, tools_for_agent, agent_prompt)
        executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)
        out = executor.invoke({"input": "What's 17*19, then fetch 2 recent AI articles."})
        print("Agent output:\n", out.get("output"))
    except Exception as e:
        print("OpenAI tools agent execution failed:", e)
else:
    print("Tool-calling agent path not available in this environment.")

## 8) Observability: Log Tool Calls and Durations

In [ ]:
def log_tool_call(fn):
    def wrapper(*args, **kwargs):
        t0 = time.time()
        res = fn(*args, **kwargs)
        dt = (time.time() - t0) * 1000
        print(f"[tool] {fn.__name__} took {dt:.1f} ms")
        return res
    return wrapper

calculator_logged = log_tool_call(calculator.func)
print("calculator 2+2:", calculator_logged("2+2"))

## 9) Exercises (Scaffolds)

A) Unit Converter Tool
- Implement unit_convert(value: float, from_unit: str, to_unit: str) with km<->miles, C<->F
- Compose with calculator; agent decides which to use

In [ ]:
from dataclasses import dataclass

@tool
def unit_convert(value: float, from_unit: str, to_unit: str) -> str:
    """Convert simple units: supports miles<->km and C<->F (case-insensitive)."""
    conv = {
        ("miles", "km"): 1.60934,
        ("km", "miles"): 1/1.60934,
    }
    try:
        fu, tu = from_unit.strip().lower(), to_unit.strip().lower()
        if (fu, tu) in conv:
            return str(value * conv[(fu, tu)])
        # Celsius <-> Fahrenheit
        if fu == "c" and tu == "f":
            return str((value * 9/5) + 32)
        if fu == "f" and tu == "c":
            return str((value - 32) * 5/9)
        return "convert-error: unsupported units"
    except Exception as e:
        return f"convert-error: {e}"

print("unit_convert tool ready.")
print("150 miles -> km:", unit_convert.func(150, "miles", "km"))

B) GitHub Repo Info Tool
- Use GitHub API to fetch repo stars/forks for "owner/repo"; add timeout and retry (scaffold below)

In [ ]:
class RepoInput(BaseModel):
    full_name: str = Field(..., description="Repository full name, e.g., 'owner/repo'")

def fetch_repo(full_name: str) -> dict:
    try:
        owner, repo = full_name.split("/")
    except ValueError:
        return {"error": "invalid format; expected 'owner/repo'"}
    url = f"https://api.github.com/repos/{owner}/{repo}"
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()
        return {"full_name": data.get("full_name"), "stars": data.get("stargazers_count"), "forks": data.get("forks_count")}
    except Exception as e:
        return {"error": str(e)}

repo_tool = StructuredTool.from_function(
    func=fetch_repo,
    name="repo_info",
    description="Fetch GitHub repo stars and forks.",
    args_schema=RepoInput
)

print(repo_tool.invoke({"full_name": "langchain-ai/langchain"}))

C) Web Retriever Tool (Async)
- Given a URL list, fetch titles and first 200 chars concurrently; summarize with LLM (scaffold only)

D) Safe File Reader
- Restrict to ./notes; reject paths with '..' or absolute; test against path traversal attempts

E) Rate-Limited News Tool
- Add simple per-minute counter; when exceeded, return a polite error

## Summary
You implemented function and structured tools, integrated external APIs and local resources, built async tools, added retries and observability, and composed tools inside an OpenAI tool-calling agent. Extend the scaffolds to complete the exercises and tailor tools for your applications.